# Test the speed of reading GFS data with 0.25 and 1.00 resolution degree into `Xarray dataset` from two sources: `NOMADS server` and `Azure Blob Storage` in Azure Databricks environment.

---

In [0]:
!python -m pip install --upgrade pip
!pip install xarray[complete]
!pip install eccodes
!pip install ecmwflibs
!pip install cfgrib
!pip install numpy==1.23.0
!pip install s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
dbutils.library.restartPython()

In [0]:
import xarray as xr
import urllib.request
import fsspec
import s3fs

### 0.25 resolution degree = 515 MB / file 
- NOMADS: 27s
- Azure Blob: 5s
- AWS S3 Bucket: 18s

In [0]:
URL = "https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20240801/12/atmos/gfs.t12z.pgrb2.0p25.f108"
filename, _ = urllib.request.urlretrieve(URL)

ds = xr.open_dataset(
        filename,
        engine="cfgrib",
        filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
        backend_kwargs={"errors": "ignore"}
)

ds

<xarray.Dataset> Size: 21MB
Dimensions:                  (latitude: 721, longitude: 1440)
Coordinates:
    time                     datetime64[ns] 8B ...
    step                     timedelta64[ns] 8B ...
    pressureFromGroundLayer  float64 8B ...
  * latitude                 (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude                (longitude) float64 12kB 0.0 0.25 ... 359.5 359.8
    valid_time               datetime64[ns] 8B ...
Data variables:
    t                        (latitude, longitude) float32 4MB ...
    r                        (latitude, longitude) float32 4MB ...
    q                        (latitude, longitude) float32 4MB ...
    u                        (latitude, longitude) float32 4MB ...
    v                        (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
URL = "https://noaagfs.blob.core.windows.net/gfs/gfs.20240801/12/atmos/gfs.t12z.pgrb2.0p25.f108"

filename, _ = urllib.request.urlretrieve(URL)
ds = xr.open_dataset(
        filename,
        engine="cfgrib",
        filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
        backend_kwargs={"errors": "ignore"}
)

ds

<xarray.Dataset> Size: 21MB
Dimensions:                  (latitude: 721, longitude: 1440)
Coordinates:
    time                     datetime64[ns] 8B ...
    step                     timedelta64[ns] 8B ...
    pressureFromGroundLayer  float64 8B ...
  * latitude                 (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude                (longitude) float64 12kB 0.0 0.25 ... 359.5 359.8
    valid_time               datetime64[ns] 8B ...
Data variables:
    t                        (latitude, longitude) float32 4MB ...
    r                        (latitude, longitude) float32 4MB ...
    q                        (latitude, longitude) float32 4MB ...
    u                        (latitude, longitude) float32 4MB ...
    v                        (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
URL = "simplecache::s3://noaa-gfs-bdp-pds/gfs.20240801/12/atmos/gfs.t12z.pgrb2.0p25.f108"
Remote_file = fsspec.open_local(URL, 
                         s3 = {'anon' : True}, 
                         filecache = {'cache_storage':'/tmp/files'})

GFS_Remote_Grib_ds = xr.open_dataset(Remote_file, 
                                     engine="cfgrib",
                                     filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
                                     backend_kwargs={"errors": "ignore"})

GFS_Remote_Grib_ds

<xarray.Dataset> Size: 21MB
Dimensions:                  (latitude: 721, longitude: 1440)
Coordinates:
    time                     datetime64[ns] 8B ...
    step                     timedelta64[ns] 8B ...
    pressureFromGroundLayer  float64 8B ...
  * latitude                 (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude                (longitude) float64 12kB 0.0 0.25 ... 359.5 359.8
    valid_time               datetime64[ns] 8B ...
Data variables:
    t                        (latitude, longitude) float32 4MB ...
    r                        (latitude, longitude) float32 4MB ...
    q                        (latitude, longitude) float32 4MB ...
    u                        (latitude, longitude) float32 4MB ...
    v                        (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:07 GRIB to CDM+CF via cfgrib-0.9.1...

### 1.00 resolution degree = 42.5 MB / file 
- NOMADS: 4s
- Azure Blob: 1s
- AWS S3 Bucket: 3s

In [0]:
URL = "https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20240801/12/atmos/gfs.t12z.pgrb2.1p00.f108"

filename, _ = urllib.request.urlretrieve(URL)
ds = xr.open_dataset(
        filename,
        engine="cfgrib",
        filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
        backend_kwargs={"errors": "ignore"}
)

ds

<xarray.Dataset> Size: 1MB
Dimensions:                  (latitude: 181, longitude: 360)
Coordinates:
    time                     datetime64[ns] 8B ...
    step                     timedelta64[ns] 8B ...
    pressureFromGroundLayer  float64 8B ...
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               datetime64[ns] 8B ...
Data variables:
    t                        (latitude, longitude) float32 261kB ...
    r                        (latitude, longitude) float32 261kB ...
    q                        (latitude, longitude) float32 261kB ...
    u                        (latitude, longitude) float32 261kB ...
    v                        (latitude, longitude) float32 261kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
URL = 'https://noaagfs.blob.core.windows.net/gfs/gfs.20240801/12/atmos/gfs.t12z.pgrb2.1p00.f108'

filename, _ = urllib.request.urlretrieve(URL)
ds = xr.open_dataset(
        filename,
        engine="cfgrib",
        filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
        backend_kwargs={"errors": "ignore"}
)

ds

<xarray.Dataset> Size: 1MB
Dimensions:                  (latitude: 181, longitude: 360)
Coordinates:
    time                     datetime64[ns] 8B ...
    step                     timedelta64[ns] 8B ...
    pressureFromGroundLayer  float64 8B ...
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               datetime64[ns] 8B ...
Data variables:
    t                        (latitude, longitude) float32 261kB ...
    r                        (latitude, longitude) float32 261kB ...
    q                        (latitude, longitude) float32 261kB ...
    u                        (latitude, longitude) float32 261kB ...
    v                        (latitude, longitude) float32 261kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
URL = "simplecache::s3://noaa-gfs-bdp-pds/gfs.20240801/12/atmos/gfs.t12z.pgrb2.1p00.f108"
Remote_file = fsspec.open_local(URL, 
                         s3 = {'anon' : True}, 
                         filecache = {'cache_storage':'/tmp/files'})

GFS_Remote_Grib_ds = xr.open_dataset(Remote_file, 
                                     engine="cfgrib",
                                     filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
                                     backend_kwargs={"errors": "ignore"})

GFS_Remote_Grib_ds

<xarray.Dataset> Size: 1MB
Dimensions:                  (latitude: 181, longitude: 360)
Coordinates:
    time                     datetime64[ns] 8B ...
    step                     timedelta64[ns] 8B ...
    pressureFromGroundLayer  float64 8B ...
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               datetime64[ns] 8B ...
Data variables:
    t                        (latitude, longitude) float32 261kB ...
    r                        (latitude, longitude) float32 261kB ...
    q                        (latitude, longitude) float32 261kB ...
    u                        (latitude, longitude) float32 261kB ...
    v                        (latitude, longitude) float32 261kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
import os
import warnings
warnings.filterwarnings('ignore')

def file_path_azure(cycle_runtime: int, forecast_hour: int, year: int, month: int, day: int, resolution_degree: float) -> str:
    prefix_path = "https://noaagfs.blob.core.windows.net/"
    product_name = "gfs"

    resolution_split = str(resolution_degree).split(".")

    file_path = (
        f"{product_name}/{product_name}.{year}{month:>02}{day:>02}/"
        f"{cycle_runtime:>02}/atmos/{product_name}.t{cycle_runtime:>02}z."
        f"pgrb2.{resolution_split[0]}p{resolution_split[1]:<02}.f{forecast_hour:>03}"
    )

    whole_path = os.path.join(prefix_path, file_path)

    return whole_path

In [0]:
def file_path_nomads(cycle_runtime: int, forecast_hour: int, year: int, month: int, day: int, resolution_degree: float) -> str:
    prefix_path = "https://nomads.ncep.noaa.gov/pub/data/nccf/com/"
    product_name = "gfs"
    version = "prod"

    resolution_split = str(resolution_degree).split(".")

    file_path = (
        f"{product_name}/{version}/{product_name}.{year}{month:>02}{day:>02}/"
        f"{cycle_runtime:>02}/atmos/{product_name}.t{cycle_runtime:>02}z."
        f"pgrb2.{resolution_split[0]}p{resolution_split[1]:<02}.f{forecast_hour:>03}"
    )

    whole_path = os.path.join(prefix_path, file_path)

    return whole_path

In [0]:
def file_path_aws(cycle_runtime: int, forecast_hour: int, year: int, month: int, day: int, resolution_degree: float) -> str:
    prefix_path = "simplecache::s3://noaa-gfs-bdp-pds/"
    product_name = "gfs"

    resolution_split = str(resolution_degree).split(".")

    file_path = (
        f"{product_name}.{year}{month:>02}{day:>02}/"
        f"{cycle_runtime:>02}/atmos/{product_name}.t{cycle_runtime:>02}z."
        f"pgrb2.{resolution_split[0]}p{resolution_split[1]:<02}.f{forecast_hour:>03}"
    )

    whole_path = os.path.join(prefix_path, file_path)

    return whole_path

### Concatenate 37 GFS file with 0.25 resolution degree (step: 0 - 36)
- NOMADS: 25m
- Azure Blob: 3m
- AWS S3 Bucket: 9m

In [0]:
ds_list = []
for i in range(0, 36 + 1):
    URL = file_path_nomads(cycle_runtime=12, forecast_hour=i, year=2024, month=7, day=30, resolution_degree=.25)
    filename, _ = urllib.request.urlretrieve(URL)
    ds = xr.open_dataset(
         filename,
         engine="cfgrib",
         filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
         backend_kwargs={"errors": "ignore"}
    )
    ds_list.append(ds)

ds_merged = xr.concat(ds_list,  dim='step')

ds_merged

<xarray.Dataset> Size: 768MB
Dimensions:                  (step: 37, latitude: 721, longitude: 1440)
Coordinates:
    time                     datetime64[ns] 8B 2024-07-30T12:00:00
  * step                     (step) timedelta64[ns] 296B 00:00:00 ... 1 days ...
    pressureFromGroundLayer  float64 8B 3e+03
  * latitude                 (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude                (longitude) float64 12kB 0.0 0.25 ... 359.5 359.8
    valid_time               (step) datetime64[ns] 296B 2024-07-30T12:00:00 ....
Data variables:
    t                        (step, latitude, longitude) float32 154MB 273.6 ...
    r                        (step, latitude, longitude) float32 154MB 98.0 ....
    q                        (step, latitude, longitude) float32 154MB 0.0038...
    u                        (step, latitude, longitude) float32 154MB -1.88 ...
    v                        (step, latitude, longitude) float32 154MB 11.3 ....
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T15:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
ds_list = []
for i in range(0, 36 + 1):
    URL = file_path_azure(cycle_runtime=12, forecast_hour=i, year=2024, month=7, day=30, resolution_degree=.25)
    filename, _ = urllib.request.urlretrieve(URL)
    ds = xr.open_dataset(
         filename,
         engine="cfgrib",
         filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
         backend_kwargs={"errors": "ignore"}
    )
    ds_list.append(ds)

ds_merged = xr.concat(ds_list,  dim='step')

ds_merged

<xarray.Dataset> Size: 768MB
Dimensions:                  (step: 37, latitude: 721, longitude: 1440)
Coordinates:
    time                     datetime64[ns] 8B 2024-07-30T12:00:00
  * step                     (step) timedelta64[ns] 296B 00:00:00 ... 1 days ...
    pressureFromGroundLayer  float64 8B 3e+03
  * latitude                 (latitude) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * longitude                (longitude) float64 12kB 0.0 0.25 ... 359.5 359.8
    valid_time               (step) datetime64[ns] 296B 2024-07-30T12:00:00 ....
Data variables:
    t                        (step, latitude, longitude) float32 154MB 273.6 ...
    r                        (step, latitude, longitude) float32 154MB 98.0 ....
    q                        (step, latitude, longitude) float32 154MB 0.0038...
    u                        (step, latitude, longitude) float32 154MB -1.88 ...
    v                        (step, latitude, longitude) float32 154MB 11.3 ....
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T15:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
ds_list = []
for i in range(0, 36 + 1):
    URL = file_path_aws(cycle_runtime=12, forecast_hour=i, year=2024, month=7, day=30, resolution_degree=.25)
    file = fsspec.open_local(URL, 
                            s3 = {'anon' : True}, 
                            filecache = {'cache_storage':'/tmp/files'})

    ds = xr.open_dataset(Remote_file, 
                        engine="cfgrib",
                        filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'},
                        backend_kwargs={"errors": "ignore"})
    ds_list.append(ds)

ds_merged = xr.concat(ds_list,  dim='step')

ds_merged

<xarray.Dataset> Size: 48MB
Dimensions:                  (step: 37, latitude: 181, longitude: 360)
Coordinates:
    time                     datetime64[ns] 8B 2024-08-01T12:00:00
  * step                     (step) timedelta64[ns] 296B 4 days 12:00:00 ... ...
    pressureFromGroundLayer  float64 8B 3e+03
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               datetime64[ns] 8B 2024-08-06
Data variables:
    t                        (step, latitude, longitude) float32 10MB 272.1 ....
    r                        (step, latitude, longitude) float32 10MB 91.0 .....
    q                        (step, latitude, longitude) float32 10MB 0.00322...
    u                        (step, latitude, longitude) float32 10MB 5.331 ....
    v                        (step, latitude, longitude) float32 10MB -2.97 ....
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-08-02T18:16 GRIB to CDM+CF via cfgrib-0.9.1...